# <center>Atelier Scraping</center>

<div style="text-align: center;">
    <img src="images/modulepython.png" alt="Example Image">
</div>

## Activité 1 : Construire un DataFrame des énigmes de Professeur Layton avec Selenium

###### Import des modules python

In [9]:
# Import des bibliothèques
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from PIL import Image
from io import BytesIO

###### Fiche récap des manières de localiser un objet sur une page web

###### Connexion au site et récupération des URL des énigmes

In [26]:
# create an instance of the Service object
service = Service(executable_path=ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-notifications")

# start Chrome using the service keyword
driver = webdriver.Chrome(service=service, options=chrome_options)

# On renseigne l'URL à laquelle on veut accéder
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"
driver.get(url)

In [27]:
#enigmes = driver.find_elements(By.XPATH, "//a[@href]")
# On va essayer de récupérer tous les liens des énigmes en recherchant par class
enigmes = driver.find_elements(By.CLASS_NAME ,"category-page__member-link")
liste_lien = []
for elem in enigmes:
    #On transforme les éléments web scrappé pour n'obtenir que les liens href associés
    liste_lien.append(elem.get_attribute("href"))

In [34]:
# On peut visualiser alors la liste obtenue et s'assurer qu'on obtient bien 106 liens
#print(liste_lien, len(liste_lien))

###### Construction du DataFrame et itération sur chaque page d'énigme : Titre, Numéro, Image, Enoncé, Indices, Solution

Pour réaliser une bonne itération, il faut voir

In [31]:
data = []
for lien in liste_lien :
    driver.get(lien)
    title = driver.find_elements(By.XPATH ,'//*[@id="firstHeading"]/span')
    
    num_enigme = driver.find_elements(By.XPATH ,'//*[@id="mw-content-text"]/div/div[1]/div[2]/table/tbody/tr[4]/td')
    
    image_enigme = driver.find_element(By.XPATH ,'//*[@id="mw-content-text"]/div/div[1]/div[2]/table/tbody/tr[2]/td/div/div/a/img')
    image_url = image_enigme.get_attribute('src')
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    sommaire = driver.find_element(By.XPATH,'//*[@id="mw-toc-heading"]')
    driver.execute_script("arguments[0].scrollIntoView();", sommaire)
    
    enonce_title = driver.find_element(By.XPATH ,'//*[@id="Énoncé"]')
    indice_title = driver.find_element(By.XPATH ,'//*[@id="Indices"]')
    
    time.sleep(2)
    enigme = []
    for i in range(1,5):
        enigme_part = enonce_title.find_element(By.XPATH, f'following-sibling::p[{i}]')
        if  enigme_part!=indice_title :
            enigme_partext = enigme_part.text
            enigme.append(enigme_partext)
    enigme = enigme[:-1]
    
    liste_indice = []
    title_indice = driver.find_element(By.XPATH,'//*[@id="Indices"]')
    driver.execute_script("arguments[0].scrollIntoView();", title_indice)

    indice1 = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[1]/ul/li[2]')
    indice1.click()
    indice1_texte = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[3]')
    liste_indice.append(indice1_texte.text)
    time.sleep(2)

    indice2 = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[1]/ul/li[3]')
    indice2.click()
    indice2_texte = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[4]/div/p')
    liste_indice.append(indice2_texte.text)
    time.sleep(2)

    indice3 = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[1]/ul/li[4]')
    indice3.click()
    indice3_texte = driver.find_element(By.XPATH,'//*[@id="mw-content-text"]/div/div[4]/div/div[5]/div/p')
    liste_indice.append(indice3_texte.text)

    
    reponse = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[7]')

    # Append the collected data as a dictionary
    data.append({
        'Title': title,
        'Numéro': num_enigme,
        'Image': image,
        'Enigme': enigme,
        'Indices': liste_indice,
        'Solution': reponse
    })
df = pd.DataFrame(data)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"following-sibling::p[1]"}
  (Session info: chrome=126.0.6478.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DB0E73+27667]
	(No symbol) [0x00D46B74]
	(No symbol) [0x00C41AFF]
	(No symbol) [0x00C82BFE]
	(No symbol) [0x00C82CDB]
	(No symbol) [0x00C78C91]
	(No symbol) [0x00CA3974]
	(No symbol) [0x00C78BA5]
	(No symbol) [0x00CA3BC4]
	(No symbol) [0x00CBCAC7]
	(No symbol) [0x00CA36C6]
	(No symbol) [0x00C774D1]
	(No symbol) [0x00C7804D]
	GetHandleVerifier [0x01063D03+2857635]
	GetHandleVerifier [0x010B7E6D+3202061]
	GetHandleVerifier [0x00E34674+566292]
	GetHandleVerifier [0x00E3B86C+595468]
	(No symbol) [0x00D4F964]
	(No symbol) [0x00D4C678]
	(No symbol) [0x00D4C817]
	(No symbol) [0x00D3E70E]
	BaseThreadInitThunk [0x75C07BA9+25]
	RtlInitializeExceptio

In [39]:
url = "https://professeur-layton.fandom.com/fr/wiki/1000_fois"

driver = webdriver.Chrome(service=service)
driver.get(url)
driver.execute_script("document.body.style.zoom = '25%'")
time.sleep(5)

enigme = driver.find_elements(By.XPATH, "//div[@id='mw-content-text']/p[preceding-sibling::p[contains(text(), 'Énoncé')] and following-sibling::p[contains(text(), 'Indices')]]")
print(enigme)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.61)
Stacktrace:
	GetHandleVerifier [0x00DB0E73+27667]
	(No symbol) [0x00D46B74]
	(No symbol) [0x00C41AFF]
	(No symbol) [0x00C2C878]
	(No symbol) [0x00C2C799]
	(No symbol) [0x00C43D20]
	(No symbol) [0x00CBC396]
	(No symbol) [0x00CA36C6]
	(No symbol) [0x00C774D1]
	(No symbol) [0x00C7804D]
	GetHandleVerifier [0x01063D03+2857635]
	GetHandleVerifier [0x010B7E6D+3202061]
	GetHandleVerifier [0x00E34674+566292]
	GetHandleVerifier [0x00E3B86C+595468]
	(No symbol) [0x00D4F964]
	(No symbol) [0x00D4C678]
	(No symbol) [0x00D4C817]
	(No symbol) [0x00D3E70E]
	BaseThreadInitThunk [0x75C07BA9+25]
	RtlInitializeExceptionChain [0x7712C10B+107]
	RtlClearBits [0x7712C08F+191]


## Activité 1 : Construire un DataFrame des énigmes de Professeur Layton avec BeautifulSoup

##### Première étape : Etablir une connexion avec le site que l'on veut scraper

In [71]:
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

##### Deuxième étape : Afficher le code source de la page

In [73]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs sse-rejected" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Catégorie:Énigmes | Wiki Professeur Layton | Fandom
  </title>
  <script>
   document.documentElement.className="client-js sse-rejected";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"5d4d0f42883166ab93c7506f954d9feb","wgCSPNonce":false,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Catégorie:Énigmes","wgTitle":"Énigmes","wgCurRevisionId":3248,"wgRevisionId":3248,"wgArticleId":748,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Énigme"],"wgPageContentLanguage":"fr","wgPageContentModel":"wikitext","wgRelevantPageName

##### Troisième étape : Récupérer les liens des énigmes, on sait qu'elles sont en bas de page, donc on va regarder la fin du code source et trouver un moyen de récupérer tous les liens

In [76]:
# Trouver tous les éléments <a> avec la classe "category-page__member-link"
links = soup.find_all('a', class_='category-page__member-link')

# Extraire les attributs href
hrefs = [link.get('href') for link in links]

# Afficher les hrefs
print(hrefs)
print(len(hrefs))

['/fr/wiki/1000_fois', '/fr/wiki/Affaires_de_famille', '/fr/wiki/Alchimie_en_folie_01', '/fr/wiki/Allumer_le_feu', '/fr/wiki/Arch%C3%A9o-logique', '/fr/wiki/Attaque_extraterrestre', '/fr/wiki/Balances_inexactes', '/fr/wiki/Boule_perdue_(1)', '/fr/wiki/Chalands_patients', '/fr/wiki/Chamailleries', '/fr/wiki/Chambre_forte', '/fr/wiki/Champ_d%27%C3%A9pouvantail_01', '/fr/wiki/Chasse_aux_souris', '/fr/wiki/Chat_balance...', '/fr/wiki/Cherchez_l%27%C3%A9toile', '/fr/wiki/Clown_ballonn%C3%A9', '/fr/wiki/Conflits_de_voisinage', '/fr/wiki/Contre-la-montre', '/fr/wiki/Couleurs_%C3%A9nigmatiques', '/fr/wiki/Course_hippique', '/fr/wiki/Croix_de_bois...', '/fr/wiki/D%27escale_en_escale', '/fr/wiki/D%27un_seul_trait_(1)', '/fr/wiki/D%27un_seul_trait_(2)', '/fr/wiki/Dans_le_Petri', '/fr/wiki/De_dr%C3%B4les_de_points', '/fr/wiki/Drapeaux_en_f%C3%AAte', '/fr/wiki/Dr%C3%B4le_de_sandwich', '/fr/wiki/D%C3%A9coupage', '/fr/wiki/D%C3%A9dale_num%C3%A9rique', '/fr/wiki/D%C3%AEner_aux_chandelles', '/fr/wiki/E

___________________________________________________________________________________

##### Précision sur l'importance de se pencher sur le code source 

Une manière un peu plus brutale, une fois qu'on a compris comment fonctionne le HTML est de récupérer tous les href de la page et de les filtrer 

In [77]:
href =[]
for link in soup.find_all('a',href=True):  # in html anchor/link is represented by the tag <a>
    href.append(link.get('href'))
    print(link.get('href'))

//professeur-layton.fandom.com/fr
#
https://professeur-layton.fandom.com/fr/wiki/Wiki_Professeur_Layton
https://professeur-layton.fandom.com/fr/wiki/Sp%C3%A9cial:AllPages
https://professeur-layton.fandom.com/fr/wiki/Sp%C3%A9cial:Community
https://professeur-layton.fandom.com/fr/wiki/Sp%C3%A9cial:AllMaps
/fr/Blog:Billets_récents
https://professeur-layton.fandom.com/fr/wiki/S%C3%A9rie_Professeur_Layton
https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:Jeux_Principaux
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_l%27%C3%89trange_Village
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_la_Bo%C3%AEte_de_Pandore
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_le_Destin_Perdu
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_l%27Appel_du_Spectre
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_le_Masque_des_Miracles
https://professeur-layton.fandom.com/fr/wiki/Professeur_Layton_et_l%27H%C3%A9rit

In [80]:
# Expression régulière pour les éléments qui commencent par "fr/wiki/"
regex = re.compile(r'^/fr/wiki/[^:]+$')

# Filtrer les éléments de la liste
filtered_list = [element for element in href if regex.match(element)]
filtered_list = list(set(filtered_list))
print(len(filtered_list))
print(filtered_list)

102
['/fr/wiki/Boule_perdue_(1)', '/fr/wiki/Attaque_extraterrestre', '/fr/wiki/Chat_balance...', '/fr/wiki/Randonn%C3%A9e_urbaine', '/fr/wiki/Les_trois_parapluies', '/fr/wiki/Photo_choc', '/fr/wiki/Tas_de_feuilles', '/fr/wiki/Chambre_forte', '/fr/wiki/Naufrage', '/fr/wiki/Allumer_le_feu', '/fr/wiki/L%27%C3%A2ge_de_ma_m%C3%A8re', '/fr/wiki/Alchimie_en_folie_01', '/fr/wiki/Tenir_l%27affiche', '/fr/wiki/Local_Sitemap', '/fr/wiki/Horloges_%C3%A0_billes', '/fr/wiki/Probl%C3%A8me_juteux', '/fr/wiki/Le_moindre_effort', '/fr/wiki/O%C3%B9_est_ma_maison_%3F', '/fr/wiki/%C3%89nigmes', '/fr/wiki/Affaires_de_famille', '/fr/wiki/Conflits_de_voisinage', '/fr/wiki/O%C3%B9_habite_Schrader_%3F', '/fr/wiki/Lapinothello', '/fr/wiki/1000_fois', '/fr/wiki/Tapis_vert', '/fr/wiki/Dans_le_Petri', '/fr/wiki/Partage_gourmand', '/fr/wiki/Couleurs_%C3%A9nigmatiques', '/fr/wiki/Croix_de_bois...', '/fr/wiki/Un_petit_g%C3%A2teau_%3F', '/fr/wiki/D%C3%AEner_aux_chandelles', '/fr/wiki/En_un_coup_de_balai', '/fr/wiki/Sus

##### Conclusion :

Bien souvent on va perdre de l'info, on voit qu'avec cette méthode, on a récupéré 102 liens vers les énigmes, alors qu'on en attend 106. On peut tenter d'améliorer le filtrage, et ça peut fonctionner, mais bien souvent il est préférable de prendre son temps sur la partie analyse du code source puis de se lancer dans la collecte.

_______________________

##### Quatrième étape : Récupération du titre, énoncé, image, etc. pour une énigme

Avant de lancer un code qui va récupérer autant d'infos, il faut au minimum s'assurer qu'il fonctionne pour une page. On minimise les erreurs et on maximise l'efficacité de collecte.

In [96]:
racine = "https://professeur-layton.fandom.com"

In [97]:
url = "https://professeur-layton.fandom.com/fr/wiki/D%C3%AEner_aux_chandelles"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")

In [98]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs sse-other" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Dîner aux chandelles | Wiki Professeur Layton | Fandom
  </title>
  <script>
   document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"47dc20598d49152478b91ae82ecfdc67","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Dîner_aux_chandelles","wgTitle":"Dîner aux chandelles","wgCurRevisionId":4390,"wgRevisionId":4390,"wgArticleId":1769,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Énigmes de l'Étrange Village","Énigmes","Inscrire la réponse","Résolue par Luke Trit

In [87]:
# Trouver la première balise meta avec l'attribut property égal à 'og:title'
#Titre, Numéro, Image, Enoncé, Indices, Solution
title = soup.find('meta', attrs={'property': "og:title"})
print(title.get("content"))
print("\n")

url_enigme = soup.find('meta', attrs={'property': "og:url"})
print(url_enigme.get("content"))
print("\n")

# Trouver le titre de la section "Énoncé"
enonce_title = soup.find('span', {'class': 'mw-headline', 'id': 'Énoncé'})
# Trouver le paragraphe suivant le titre de la section "Énoncé"
enonce_paragraph = enonce_title.find_next('p')
# Extraire le texte du paragraphe
enonce_text = enonce_paragraph.get_text(strip=True)
print(enonce_text)

print("\n")
reponse_title = soup.find('span', {'class': 'mw-headline', 'id': 'Solution'})
reponse_paragraph = reponse_title.find_next('p')
reponse = reponse_paragraph.get_text(strip=True)
print(reponse)


print("\n")
# Find all tab labels
tabs = soup.select('ul.wds-tabs li.wds-tabs__tab a')
contents = soup.select('div.wds-tab__content')

# Extract each index content into a list
indices = []
for i, tab in enumerate(tabs):
    if i < len(contents):
        content_divs = contents[i].select('div[style*="overflow-y:auto"]')
        if content_divs:
            content = content_divs[0].get_text(strip=True)
            indices.append(content)

# Display the extracted indices as a list
for i, content in enumerate(indices):
    print(f'Indice {i + 1}: {content}\n')
print(indices)

Dîner aux chandelles


https://professeur-layton.fandom.com/fr/wiki/D%C3%AEner_aux_chandelles


Dix bougies sont allumées dans votre salle à manger. Un courant d'air vient éteindre deux d'entre elles. Plus tard, vous vous rendez compte qu'une bougie supplémentaire s'est éteinte. Vous fermez la fenêtre afin de vous assurer qu'aucune autre bougie ne s'éteindra. En supposant que le vent ne vous perturbera plus, combien de bougies va-t-il vous rester ?


La réponse est 3.


Indice 1: Cela peut paraître évident, mais ce ne serait pas une énigme s'il n'y avait pas un petit piège. Relisez l'énoncé attentivement.On vous demande combien de bougies il va vous rester.

Indice 2: Le vent éteint deux bougies, puis une autre peu après. Combien de bougies se sont éteintes ?Les bougies qui n'ont pas été éteintes vont continuer à se consumer. Que va-t-il leur arriver si elle continuent de brûler ?

Indice 3: On vous demande combien de bougies il va vous rester. Pour cela, il faut que l'objet en questio

In [55]:
# Trouver toutes les balises avec un attribut src
src_tags = soup.find_all(src=True)

# Extraire les valeurs de src
src_urls = [tag['src'] for tag in src_tags]

# Afficher les valeurs src trouvées
#for src in src_urls:
    #print(src)
longest_src = max(src_urls, key=len) if src_urls else None
print(longest_src)
image = longest_src

https://static.wikia.nocookie.net/layton/images/8/87/EV_015.png/revision/latest/scale-to-width-down/270?cb=20160922061307&path-prefix=fr


##### Cinquième étape : Généralisation

In [112]:
def collecte_enigme(racine, href):
    url = racine+href
    data  = requests.get(url).text
    soup = BeautifulSoup(data,"html.parser")
    
    # Récupération du titre
    title = soup.find('meta', attrs={'property': "og:title"})
    title = title.get("content")

    # Récupération url
    url_enigme = soup.find('meta', attrs={'property': "og:url"})
    url_enigme = url_enigme.get("content")

    # Récupération de l'image
    src_tags = soup.find_all(src=True)
    # Extraire les valeurs de src
    src_urls = [tag['src'] for tag in src_tags]
    longest_src = max(src_urls, key=len) if src_urls else None
    image = longest_src

    # Récupération de l'énigme
    try:
        enigme_title = soup.find('span', {'class': 'mw-headline', 'id': 'Énoncé'})
        # Trouver le paragraphe suivant le titre de la section "Énoncé"
        enigme_paragraph = enigme_title.find_next('p')
        # Extraire le texte du paragraphe
        enigme = enigme_paragraph.get_text(strip=True)
        
    except :
        enigme = image


    # Récupération de la réponse
    try :
        
        reponse_title = soup.find('span', {'class': 'mw-headline', 'id': 'Solution'})
        reponse_paragraph = reponse_title.find_next('p')
        reponse = reponse_paragraph.get_text(strip=True)
    except :
        a_tag = soup.find('a', class_='image')
        if a_tag:
            reponse = a_tag.get('href')
        else:
            reponse ="Problème"

    # Récupération des indices
    tabs = soup.select('ul.wds-tabs li.wds-tabs__tab a')
    contents = soup.select('div.wds-tab__content')
    # Extract each index content into a list
    indices = []
    for i, tab in enumerate(tabs):
        if i < len(contents):
            content_divs = contents[i].select('div[style*="overflow-y:auto"]')
            if content_divs:
                content = content_divs[0].get_text(strip=True)
                indices.append(content)
        
    # Append the collected data as a dictionary
    data = []
    data.append({'Title': title, 'url': url_enigme, 'Image': image, 'Enigme': enigme, 'Indices': indices,'Solution': reponse})

    
    df = pd.DataFrame(data)
    return df

In [105]:
racine = "https://professeur-layton.fandom.com"
df_final = pd.DataFrame()
for href in hrefs:
    #print(href)
    df_final = pd.concat([df_final, collecte_enigme(racine,href)], ignore_index=True)

In [106]:
df_final

,Title,url,Image,Enigme,Indices,Solution
0,1000 fois,https://professeur-layton.fandom.com/fr/wiki/1...,https://static.wikia.nocookie.net/layton/image...,Mais qu'est-ce que cela peut bien vouloir dire...,[Il n'existe que 26 lettres en français. Si vo...,La réponse est M.
1,Affaires de famille,https://professeur-layton.fandom.com/fr/wiki/A...,https://static.wikia.nocookie.net/layton/image...,Un garçon et sa sœur discutent tranquillement ...,[Essayez de bien comprendre chaque élément de ...,La réponse est 6 pour les deux.
2,Alchimie en folie 01,https://professeur-layton.fandom.com/fr/wiki/A...,https://static.wikia.nocookie.net/layton/image...,https://static.wikia.nocookie.net/layton/image...,[],Solution à l'énigme.
3,Allumer le feu,https://professeur-layton.fandom.com/fr/wiki/A...,https://static.wikia.nocookie.net/layton/image...,Il ne vous reste plus qu'une allumette.,[Est-ce que l'ordre dans lequel vous procédez ...,"C'est évident finalement, non ?"
4,Archéo-logique,https://professeur-layton.fandom.com/fr/wiki/A...,https://static.wikia.nocookie.net/layton/image...,"Pendant les fouilles, A, B, C et D ont chacun ...",[Les données suivantes sont déjà reportées dan...,Il suffit de cocher les symboles et les nombre...
...,...,...,...,...,...,...
101,Énigmes de l'Étrange Village,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,https://static.wikia.nocookie.net/layton/image...,[],Problème
102,Énigmes de la Boîte de Pandore,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,https://static.wikia.nocookie.net/layton/image...,[],Problème
103,Énigmes du Destin Perdu,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,https://static.wikia.nocookie.net/layton/image...,[],Problème
104,Énigmes du Masque des Miracles,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,https://static.wikia.nocookie.net/layton/image...,[],Problème


# Conclusion

Nous avons finalement réussi à se constituer un DataFrame à partir d'un site web ! 
Ce qu'il faut retenir, c'est que BeautifulSoup est très utile pour les sites dit "ouverts" et pour de la répetition massive de collecte d'infos.

# Sources 
- [Le web scrping est-il légal ?](https://www.iubenda.com/fr/help/111962-le-web-scraping-est-il-legal-ce-que-vous-devez-savoir#:~:text=La%20l%C3%A9galit%C3%A9%20du%20web%20scraping&text=Ne%20soyez%20pas%20trop%20enthousiaste,pas%20prot%C3%A9g%C3%A9es%20par%20un%20login)